# Extract Data from Fight Events

So for each fight we to:
 - Get the elemental summon and waterbolt events.
 - Get the `combatantinfo` event for the summoning mage.
 - Get the buff events for the mage and elemental.

In [80]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [159]:
!pwd

/Users/shaxby/src/personal/magesim-data/notebooks


In [81]:
import sys
src = '../src/personal/magesim-data/'
sys.path.append(src)

In [82]:
# All Kara encounters
encounters = list(range(652, 663))
encounters.remove(660)

In [83]:
from wcl.auth import get_oauth_token
from wcl.query import WCLClient
from wcl.magesim.query import EVENT_ALL_QUERY

import asyncio

In [39]:
c = WCLClient(get_oauth_token())

In [175]:
MASTER_QUERY = """
    query ($zoneID: Int!, $encounterID: Int!, $startTime: Float!, $limit: Int!, $page: Int!) {
      reportData {
        reports(zoneID: $zoneID, guildServerRegion: "US", startTime: $startTime, limit: $limit, page: $page) {
          last_page
          has_more_pages
          data {
            code
            events(encounterID: $encounterID, startTime: 0, endTime: 999999999, limit: 20000) {
               data
            }
          }
        }
      }
    }
"""

In [176]:
def extract(data):
    reports = {}
    for i, report in enumerate(data):
        events_to_keep = []
        sources = {}
        mages = set()
        targets = set()
        events = report["events"]["data"]
        for event in events:
            # Save sourceID so we can grab later if they summon elemental.
            if event["type"] == "combatantinfo":
                sources[event["sourceID"]] = event
            elif event["type"] == "summon" and event["abilityGameID"] == 31687:
                events_to_keep.append(event)
                events_to_keep.append(sources[event["sourceID"]])
                targets.add(event["targetID"])
                mages.add(event["sourceID"])
            # Waterbolt
            elif event["type"] == "damage" and event["abilityGameID"] == 31707:
                events_to_keep.append(event)
            # Track buffs - especially drums and hero for elemental and mage.
            elif event["type"] in ["applybuff", "removebuff"] and (
                    event["targetID"] in targets or event["sourceID"] in mages):
                events_to_keep.append(event)

        reports[i] = events_to_keep
        
    return reports

In [177]:
encounter_id = 652
start_time = 0
limit = 10
page = 1

qparams = {
    "zoneID": 1007, # kara
    "encounterID": encounter_id,
    "startTime": start_time,
    "limit": limit,
    "page": page,
}
res = await c.query(MASTER_QUERY, qparams)
data = res["reportData"]["reports"]["data"]

In [178]:
reports = extract(data)

In [182]:
data[1]

{'code': 'rfFhAy1YpMq4VKvJ',
 'events': {'data': [{'timestamp': 173511629,
    'type': 'encounterstart',
    'encounterID': 652,
    'name': 'Attumen the Huntsman',
    'difficulty': 3,
    'size': 10},
   {'timestamp': 173511629,
    'type': 'combatantinfo',
    'sourceID': 517,
    'gear': [{'id': 31546,
      'quality': 3,
      'icon': 'inv_misc_gem_ruby_03.jpg',
      'itemLevel': 109},
     {'id': 28134,
      'quality': 3,
      'icon': 'inv_jewelry_necklace_27.jpg',
      'itemLevel': 112},
     {'id': 28967,
      'quality': 4,
      'icon': 'inv_shoulder_52.jpg',
      'itemLevel': 120,
      'gems': [{'id': 24056,
        'itemLevel': 70,
        'icon': 'inv_jewelcrafting_nightseye_03.jpg'},
       {'id': 31867,
        'itemLevel': 70,
        'icon': 'inv_jewelcrafting_nightseye_03.jpg'}]},
     {'id': 0, 'quality': 1, 'icon': 'inv_axe_02.jpg', 'itemLevel': 0},
     {'id': 28964,
      'quality': 4,
      'icon': 'inv_chest_cloth_62.jpg',
      'itemLevel': 120,
      'pe

In [148]:
reports

{0: [],
 1: [],
 2: [],
 3: [{'timestamp': 489200,
   'type': 'summon',
   'sourceID': 15,
   'targetID': 29,
   'abilityGameID': 31687},
  {'timestamp': 427211,
   'type': 'combatantinfo',
   'sourceID': 15,
   'gear': [{'id': 28744,
     'quality': 4,
     'icon': 'inv_helmet_53.jpg',
     'itemLevel': 115,
     'permanentEnchant': 3002},
    {'id': 28134,
     'quality': 3,
     'icon': 'inv_jewelry_necklace_27.jpg',
     'itemLevel': 112},
    {'id': 21869,
     'quality': 4,
     'icon': 'inv_shoulder_25.jpg',
     'itemLevel': 105,
     'gems': [{'id': 30564,
       'itemLevel': 70,
       'icon': 'inv_jewelcrafting_nightseye_03.jpg'},
      {'id': 30605,
       'itemLevel': 70,
       'icon': 'inv_jewelcrafting_talasite_03.jpg'}]},
    {'id': 0, 'quality': 1, 'icon': 'inv_axe_02.jpg', 'itemLevel': 0},
    {'id': 28342,
     'quality': 3,
     'icon': 'inv_chest_cloth_29.jpg',
     'itemLevel': 115,
     'permanentEnchant': 2661,
     'gems': [{'id': 28461,
       'itemLevel': 55

In [163]:
data_dir = "/Users/shaxby/src/personal/magesim-data/data"
extract_subdir = Path(data_dir, "extracts", "v1", str(encounter_id))

In [171]:
extract_subdir.as_posix()

'/Users/shaxby/src/personal/magesim-data/data/extracts/v1/652'

In [174]:
import json
import os
from pathlib import Path

data_dir = "/Users/shaxby/src/personal/magesim-data/data"
extract_subdir = Path(data_dir, "extracts", "v1", str(encounter_id))
os.makedirs(extract_subdir.as_posix())

for num, report in reports.items():
    if report:
        fp = os.path.join(extract_subdir.as_posix(), f"report_{num}.json")
        with open(fp, "w") as fobj:
            json.dump(report, fobj)

### Reduce # of events we're pulling?
Let see if we can query more intelligently to filter out reports.

In [120]:
MASTER_QUERY_V1 = """
    query ($zoneID: Int!, $encounterID: Int!, $startTime: Float!, $limit: Int!, $page: Int!) {
      reportData {
        reports(zoneID: $zoneID, guildServerRegion: "US", startTime: $startTime, limit: $limit, page: $page) {
          last_page
          has_more_pages
          data {
            events(sourceClass: "Mage", encounterID: $encounterID, startTime: 0, endTime: 999999999, limit: 20000) {
               data
            }
          }
        }
      }
    }
"""
test_res = await c.query(MASTER_QUERY_V1, qparams)
test_data = test_res["reportData"]["reports"]["data"]

In [121]:
test_reports = extract(test_data)

In [122]:
test_reports

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

Doesn't seem like it -- https://classic.warcraftlogs.com/v2-api-docs/warcraft/report.doc.html.

We can do mono-filtering on a single entity / event type but we need multiple event types with multiple source / targets.